In [ ]:
import cv2
import gradio as gr

# Haar Cascade 파일 로드
cascPath = 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

def mosaic_face(image):
    if image is None:
        return None

    # Gradio는 RGB로 들어옴 → BGR로 변환
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
    )

    for (x, y, w, h) in faces:
        face_roi = image[y:y+h, x:x+w]

        # 모자이크 처리
        mosaic = cv2.resize(face_roi, (10, 10), interpolation=cv2.INTER_LINEAR)
        mosaic = cv2.resize(mosaic, (w, h), interpolation=cv2.INTER_NEAREST)

        image[y:y+h, x:x+w] = mosaic

    # 다시 RGB로 변환해서 Gradio로 리턴
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Gradio 인터페이스
with gr.Blocks() as demo:
    with gr.Row():
        webcam = gr.Image(sources="webcam", streaming=True, type="numpy", label="Webcam Input")
        output = gr.Image(label="Mosaic Output", streaming=True)

    webcam.stream(fn=mosaic_face, inputs=webcam, outputs=output)

demo.launch()
